In [1]:
from pyscipopt import Model, quicksum, multidict
from cflp_function import *
from calculate_od import *
import pydeck as pdk
from pydeck.types import String

In [2]:
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df
def load_gdf(gdf_path):
    gdf = gpd.read_file(gdf_path)
    return gdf

In [3]:
main_crs = "EPSG:4326"

In [4]:
farm = load_csv("./farm/farm_mock.csv")
farm['color'] = '[169, 169, 169]'

In [6]:
folder_path = "app_data"
J = load_data_from_pickle(folder_path, 'Farm_test.pickle')
M = load_data_from_pickle(folder_path, 'manure_production_test.pickle')

In [7]:
def calculate_od_matrix_new(farm_gdf, loi_gdf, cost_per_km=0.69, frequency_per_day=1, lifetime_in_days=1):
    """
    A function to find the nearest road network node for each candidate site.

    Parameters
    ----------
    farm_gdf : GeoDataFrame
        GeoDataFrame of farm points. 
    loi_gdf : GeoDataFrame
        Geodataframe of candidate sites.
    cost_per_km = int/float, optional
        Unit cost for transporting feedstocks from sources to digesters. 

    Outputs
    ----------
    c : dict
        Dictionary of OD matrix {} 
    plant : list
        List of indices of candidate digester sites

    """
    g = ox.load_graphml('./osm_network/G.graphml') 
    orig = farm_gdf['closest_osmid'].unique().tolist()
    dest = loi_gdf['closest_osmid'].unique().tolist()

    # Initialize an empty OD matrix
    od_matrix = {}

    # Calculate shortest path between all pair orig (farm) and dest (set of candidate digester sites)
    for origin in orig:
        od_matrix[origin] = {}
        for destination in dest:
            distance = nx.shortest_path_length(g, origin, destination, weight='length')
            od_matrix[origin][destination] = distance/1000 # convert from m to km
            # output dict = {orig:{dest:distance, dest:distance....}}

    # Initialize an empty nested dictionary
    new_nested_dict = {}
    # Create a new nested dictionary with DataFrame indices as keys {farm1:{dest_node_1:distance, dest_node_2:distance....}} 
    # Some road network nodes are the closest for more than 1 farms, so now we make sure the dictionary has a key of all farms despite some wil linherit the  
    # same associated distances to all digesters. This dictionary structure is required for the optimization model later.
    for idx, row in farm_gdf.iterrows():
        osmid_value = row['closest_osmid']
        if osmid_value in od_matrix:
            new_nested_dict[idx] = od_matrix[osmid_value]    

    # A placeholder that maps digester candidate site index with the index of its closest node
    placeholders = {i:j for i, j in zip(loi_gdf.index.values, loi_gdf['closest_osmid'])}

    restructured_od = {}
    for farm, distances in new_nested_dict.items():
        restructured_od[farm] = {}
        for index, placeholder in placeholders.items():
            restructured_od[farm][index] = distances.get(placeholder, None)

    new_dict = {}
    for farm, digester_distances in restructured_od.items():
        for digester, distance in digester_distances.items():
            new_key = (digester, farm)
            new_dict[new_key] = distance
   
    transport_cost = dict(sorted(new_dict.items(), key=lambda x: x[0][0]))
    # transport_cost = {(index, farm): distance for index, farms in restructured_od.items() for farm, distance in farms.items()}

    # Convert from distance to cost
    c = {key: value * cost_per_km * frequency_per_day * lifetime_in_days for key, value in transport_cost.items()}
    plant = loi_gdf.index.tolist()

    # store_data_to_pickle(transport_cost, 'app_data', transport_cost_file_name)
    # store_data_to_pickle(loi_gdf.index.tolist(), 'app_data', plant_file_name)

    return c, plant

In [8]:
### LOAD DATA ###
loi = load_csv('./hex/loi.csv') # Mock candidate sites
farm_gdf = load_gdf("./farm/farm_new.shp")
n = load_gdf("./osm_network/G_n.shp") # Road network nodes
n = n.to_crs(main_crs)
### CALCULATE OD MATRIX ###
loi.index = range(1, len(loi) +1) # Reset index to start with 1 (because users don't like 0 as the 1st value...)
loi_gdf = loi_to_gdf(loi) # Find centroid of hexagons and convert to gdf
loi_gdf['y'] = loi_gdf['geometry'].y
loi_gdf['x'] = loi_gdf['geometry'].x
find_closest_osmid(farm_gdf, n)
find_closest_osmid(loi_gdf, n)

In [9]:
C, plant = calculate_od_matrix_new(farm_gdf, loi_gdf, cost_per_km=0.69, frequency_per_day=1, lifetime_in_days=1)

In [10]:
d, f = assign_capacity_capex(plant)
I = plant

In [11]:
percentage = 0.6

In [12]:
# New version 
model = Model("flp_v2")

X,y = {},{}

# # Express the total manure in the region & target demand input by users
total_demand = sum(M[j] for j in J)
target_demand = total_demand * percentage

# Decision variables
for i in I:
    y[i] = model.addVar(vtype="B", name="y(%s)"%i)
    for j in J:
        X[i,j] = model.addVar(vtype="C", name="X(%s,%s)"%(i,j))

# Constraint 1
for i in I:
    model.addCons(quicksum(X[i,j] for j in J) <= d[i]*y[i], "Demand_(of_Digester_Capacity)(%s)"%i)
# Constrain 2
for j in M:
    model.addCons(quicksum(X[i,j] for i in I) <= M[j], "Capacity_(of_Farm_Manure_Production)(%s)"%i)
# Constrain 3
for (i,j) in X:
    model.addCons(X[i,j] <= d[i]*y[i], "Strong(%s,%s)"%(i,j))
# Constraint 4
model.addCons(quicksum(X[i, j] for i in I for j in J) >= target_demand, "TargetManureDemand")

# Objective function
model.setObjective(
    quicksum(f[i]*y[i] for i in I) +
    quicksum(C[i,j]*X[i,j] for i in I for j in J),
    "minimize")
model.data = X,y

In [13]:
# model = Model("flp_percentage_demand")

# x, y, z = {}, {}, {}
# total_demand = sum(d[i] for i in I)
# target_demand = total_demand * p

# for j in J:
#     y[j] = model.addVar(vtype="B", name="y(%s)" % j)
#     for i in I:
#         x[i, j] = model.addVar(vtype="C", name="x(%s,%s)" % (i, j))
#         z[i, j] = model.addVar(vtype="B", name="z(%s,%s)" % (i, j))
# # # Enforce z[i, j] = 1 if x[i, j] > 0
# # for (i, j) in x:
# #     model.addCons(z[i, j] >= x[i, j] / d[i], "EnforceZCondition(%s,%s)" % (i, j))

# for i in I:
#     model.addCons(quicksum(x[i, j] for j in J) == d[i] * z[i, j], "Demand(%s)" % i)

# for j in M:
#     model.addCons(quicksum(x[i, j] for i in I) <= M[j] * y[j], "Capacity(%s)" % j)

# for (i, j) in x:
#     model.addCons(x[i, j] <= d[i] * y[j], "Strong(%s,%s)" % (i, j))

# model.addCons(quicksum(x[i, j] for i in I for j in J) >= target_demand, "PercentageDemand")

# model.setObjective(
#     quicksum(f[j] * y[j] for j in J) +
#     quicksum(c[i, j] * z[i, j] for i in I for j in J),
#     "minimize"
# )
# model.data = x, y, z

In [14]:
model.optimize()

In [15]:
model.getObjVal()

77435759.56279054

In [16]:
EPS = 1.e-6
x,y = model.data
assignment = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [i for i in y if model.getVal(y[i]) > EPS]

In [17]:
facilities

[3, 4, 6, 7]

In [19]:
# Create a dictionary to store the results
result_dict = {f: [] for f in facilities}
# Iterate over edges and populate the result_dict
for (i, j) in assignment:
    if i in facilities:
        result_dict[i].append(j)

In [21]:
# total_c = sum(c[key] for key in assignment if key in c)*365*12

In [ ]:
# Example dictionary
result_dict = {(1, 2): 10, (3, 4): 20, (5, 6): 30}

# Example values
total_capex = len(result_dict) * 6089160
total_opex = len(result_dict) * (1047200 * 12)

# Sum the values corresponding to the keys in result_dict
total_c = sum(result_dict[key] for key in result_dict)

# Create a DataFrame
data = {'Category': ['Total C', 'Total Capex', 'Total Opex'],
        'Value': [total_c, total_capex, total_opex]}

df = pd.DataFrame(data)

# Display the DataFrame
print(df)

      Category     Value
0      Total C        60
1  Total Capex  18267480
2   Total Opex  37699200


In [ ]:
total_cost = model.getObjVal()

# Create a dictionary to store the results
result_dict = {f: [] for f in facilities}
# Iterate over edges and populate the result_dict
for (i, j) in assignment:
    if j in facilities:
        result_dict[j].append(i)

# Get percentage of utilization
x_values = {(i, j): model.getVal(x[i, j]) for (i, j) in x if model.getVal(x[i, j]) > EPS} # get how much is flowing between every assignment
flow_matrix = np.array([[x_values.get((i, j), 0) for j in J] for i in I]) # create a flow matrix (len(farm)xlen(plant))
column_sum = np.sum(flow_matrix, axis=0) # sum of total flow going to every plant
used_capacity = (column_sum/np.array(list(M.values())))*100

In [ ]:
used_capacity_df = pd.DataFrame(used_capacity, index=J)

In [22]:
arc_layer_df = get_arc(result_dict, loi_gdf, farm)

In [23]:
digester_layer = pdk.Layer(type='ScatterplotLayer',
                        data=loi_gdf,
                        get_position=['x', 'y'],
                        get_radius=800,
                        get_fill_color='color',
                        pickable=True,
                        auto_highlight=True, 
                        get_line_color=[255, 255, 255],
                        get_line_width=3)
farm_layer = pdk.Layer(type='ScatterplotLayer',
                    data=farm,
                    get_position=['x', 'y'],
                    get_radius=300,
                                get_fill_color='color',
                                get_line_color=[0, 0, 0],
                                pickable=False,
                                auto_highlight=True)
loi_gdf['name'] = loi_gdf.index.astype(str)
digester_label_layer = pdk.Layer(
    "TextLayer",
    loi_gdf,
    pickable=True,
    get_position=['x', 'y'],
    get_text="name",
    get_size=18,
    get_color=[255,255,255],
    get_angle=0,
# Note that string constants in pydeck are explicitly passed as strings
# This distinguishes them from columns in a data set
get_text_anchor=String("middle"),
get_alignment_baseline=String("center"))


arc_layer = pdk.Layer(
    'ArcLayer',
    data=arc_layer_df,
    get_width=2,          # Width of the arcs
    get_source_position=['start_lon', 'start_lat'],
    get_target_position=['end_lon', 'end_lat'],
    get_source_color=[0, 255, 0, 160],   # RGBA color of the starting points
    get_target_color=[255, 0, 0, 160],   # RGBA color of the ending points
    pickable=True,
    auto_highlight=True
)
view_state=pdk.ViewState(
    latitude=loi_gdf['y'].mean(),
    longitude=loi_gdf['x'].mean(),
    zoom=9,
    )
deck = pdk.Deck(
    layers=[farm_layer, digester_layer, digester_label_layer, arc_layer],
    initial_view_state=view_state, 
    map_style= 'mapbox://styles/mapbox/outdoors-v12'
    )
deck.to_html('optimization.html')
